# 시장데이터 종가 생성

1. col_62 파일 기반으로 재생성
2. 9920종가 파일 기반 - 99년 데이터부터 사용함으로서 정확도를 높임


In [68]:
import pandas as pd
import numpy as np
from tqdm import tqdm 

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
plt.rcParams['font.family'] = 'malgun gothic'

from sklearn.metrics import accuracy_score, log_loss, plot_confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score,average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier, Pool  
from lightgbm import LGBMClassifier   

from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler

import warnings
warnings.filterwarnings(action='ignore')

## 데이터 불러오기

In [69]:
del price

In [70]:
price = pd.read_csv(r'..\..\..\..\Code\SH\csv\9920종가.csv', encoding= 'euc-kr')
price_original = price

In [71]:
price

,회사명,거래소코드,회계년도,종가
0,(주)CMG제약,58820,2000/12,NaN
1,(주)CMG제약,58820,2001/12,1420.0
2,(주)CMG제약,58820,2002/12,520.0
3,(주)CMG제약,58820,2003/12,800.0
4,(주)CMG제약,58820,2004/12,550.0
...,...,...,...,...
27936,흥구석유(주),24060,2020/12,7220.0
27937,흥구석유(주),24060,2000/12,265000.0
27938,흥구석유(주),24060,2011/12,1500.0
27939,흥구석유(주),24060,2012/12,1585.0


In [72]:
price.isna().sum()

회사명         0
거래소코드       0
회계년도        0
종가       6807
dtype: int64

In [6]:
#일단 결측치 fillna로 0채워줌
#price = price.fillna(0)

## key 생성
거래소코드+년도

In [73]:
price['회계년도'].str[:4]

0        2000
1        2001
2        2002
3        2003
4        2004
         ... 
27936    2020
27937    2000
27938    2011
27939    2012
27940    2008
Name: 회계년도, Length: 27941, dtype: object

In [74]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27941 entries, 0 to 27940
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사명     27941 non-null  object 
 1   거래소코드   27941 non-null  int64  
 2   회계년도    27941 non-null  object 
 3   종가      21134 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 873.3+ KB


In [75]:
price['년'] = price['회계년도'].str[:4].astype(int)

In [76]:
price['년'] = price['회계년도'].str[:4].astype(int)
price['key'] = price['거래소코드'].astype(str) + (price['년']+1).astype(str)

In [77]:
price

,회사명,거래소코드,회계년도,종가,년,key
0,(주)CMG제약,58820,2000/12,NaN,2000,588202001
1,(주)CMG제약,58820,2001/12,1420.0,2001,588202002
2,(주)CMG제약,58820,2002/12,520.0,2002,588202003
3,(주)CMG제약,58820,2003/12,800.0,2003,588202004
4,(주)CMG제약,58820,2004/12,550.0,2004,588202005
...,...,...,...,...,...,...
27936,흥구석유(주),24060,2020/12,7220.0,2020,240602021
27937,흥구석유(주),24060,2000/12,265000.0,2000,240602001
27938,흥구석유(주),24060,2011/12,1500.0,2011,240602012
27939,흥구석유(주),24060,2012/12,1585.0,2012,240602013


## 종가변동률 구하기

In [11]:
#종가변동률 구하는 식 <br>
#(df['자기자본배율'] - df['자기자본배율'].shift(1))/df['자기자본배율'].shift(1)*100

In [78]:
# for문으로 생성해주기
#종가변동률 구하는 식 <br>
#(df['자기자본배율'] - df['자기자본배율'].shift(1))/df['자기자본배율'].shift(1)*100
company = price['회사명'].unique().tolist()

for i in tqdm(range(len(company))):
    price.loc[price['회사명']==company[i], '종가변동률']=(price.loc[price['회사명']==company[i]]['종가'] - price.loc[price['회사명']==company[i]]['종가'].shift(1))/price.loc[price['회사명']==company[i]]['종가'].shift(1)
    #price = price.fillna(0)
    price['종가변동률'] = price['종가변동률'].replace([np.inf], np.nan)
    price['종가변동률'] = price['종가변동률'].fillna(method='bfill')# 후년 데이터로 대체
    

100%|██████████| 2051/2051 [00:14<00:00, 142.65it/s]


In [79]:
price

,회사명,거래소코드,회계년도,종가,년,key,종가변동률
0,(주)CMG제약,58820,2000/12,NaN,2000,588202001,-0.633803
1,(주)CMG제약,58820,2001/12,1420.0,2001,588202002,-0.633803
2,(주)CMG제약,58820,2002/12,520.0,2002,588202003,-0.633803
3,(주)CMG제약,58820,2003/12,800.0,2003,588202004,0.538462
4,(주)CMG제약,58820,2004/12,550.0,2004,588202005,-0.312500
...,...,...,...,...,...,...,...
27936,흥구석유(주),24060,2020/12,7220.0,2020,240602021,0.347015
27937,흥구석유(주),24060,2000/12,265000.0,2000,240602001,35.703601
27938,흥구석유(주),24060,2011/12,1500.0,2011,240602012,-0.994340
27939,흥구석유(주),24060,2012/12,1585.0,2012,240602013,0.056667


In [80]:
price.loc[price['회사명']==company[10]]

,회사명,거래소코드,회계년도,종가,년,key,종가변동률
130,(주)고영테크놀러지,98460,2005/12,NaN,2005,984602006,0.530758
131,(주)고영테크놀러지,98460,2006/12,NaN,2006,984602007,0.530758
132,(주)고영테크놀러지,98460,2007/12,NaN,2007,984602008,0.530758
133,(주)고영테크놀러지,98460,2008/12,6990.0,2008,984602009,0.530758
134,(주)고영테크놀러지,98460,2009/12,10700.0,2009,984602010,0.530758
135,(주)고영테크놀러지,98460,2010/12,18000.0,2010,984602011,0.682243
136,(주)고영테크놀러지,98460,2011/12,30900.0,2011,984602012,0.716667
137,(주)고영테크놀러지,98460,2012/12,28100.0,2012,984602013,-0.090615
138,(주)고영테크놀러지,98460,2013/12,27950.0,2013,984602014,-0.005338
139,(주)고영테크놀러지,98460,2014/12,42800.0,2014,984602015,0.531306


In [81]:
# 결측치 확인
price.isna().sum()

회사명         0
거래소코드       0
회계년도        0
종가       6807
년           0
key         0
종가변동률       0
dtype: int64

In [16]:
#csv저장
#price.to_csv('9920종가변동률 반영.csv')

## finaldataset과 merge

In [2]:
#price = pd.read_csv(r'..\..\..\Code\SH\4. 텍스트마이닝\0020종가변동률 반영.csv')

In [17]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '상장폐지일', '감사의견코드', '자기자본배당률',
       '금융비용 대 부채비율', '자기자본배율', '유동부채비율', '매입채무회전률', '유동자산회전률', '총자본투자효율',
       '순운전자본비율', '누적수익성비율', '총자산영업이익률', '총자산회전율', '년', 'key', '부도', '공시발생일',
       '정기공시제목', '수시공시제목', '대표이사 변경', '최대주주 변경', '회계처리위반', '횡령배임', '신종채권',
       '영업조업중단', '출자목적_투자', '출자목적_경영권', '출자목적_영업이익', '기타'],
      dtype='object')

In [82]:
df_ = pd.read_csv(r'..\..\..\..\Code\SH\1. 데이터생성\최종\col_62.csv', index_col=0)
df = df_

In [83]:
df.shape

(27357, 69)

In [84]:
df['key']

0        588202001
1        588202002
2        588202003
3        588202004
4        588202005
           ...    
27352    240602017
27353    240602018
27354    240602019
27355    240602020
27356    240602021
Name: key, Length: 27357, dtype: int64

In [85]:
price['key']

0        588202001
1        588202002
2        588202003
3        588202004
4        588202005
           ...    
27936    240602021
27937    240602001
27938    240602012
27939    240602013
27940    240602009
Name: key, Length: 27941, dtype: object

In [86]:
price[['key','종가변동률']]

,key,종가변동률
0,588202001,-0.633803
1,588202002,-0.633803
2,588202003,-0.633803
3,588202004,0.538462
4,588202005,-0.312500
...,...,...
27936,240602021,0.347015
27937,240602001,35.703601
27938,240602012,-0.994340
27939,240602013,0.056667


In [87]:
df['key'] = df['key'].astype(str)

In [88]:
df['key']

0        588202001
1        588202002
2        588202003
3        588202004
4        588202005
           ...    
27352    240602017
27353    240602018
27354    240602019
27355    240602020
27356    240602021
Name: key, Length: 27357, dtype: object

In [89]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27941 entries, 0 to 27940
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사명     27941 non-null  object 
 1   거래소코드   27941 non-null  int64  
 2   회계년도    27941 non-null  object 
 3   종가      21134 non-null  float64
 4   년       27941 non-null  int32  
 5   key     27941 non-null  object 
 6   종가변동률   27941 non-null  float64
dtypes: float64(2), int32(1), int64(1), object(3)
memory usage: 1.4+ MB


In [104]:
merged[merged['거래소코드'] == 66110]

,회사명,거래소코드,회계년도,소속코드,상장폐지일,유동비율,당좌비율,유동부채비율,비유동비율,순운전자본비율,...,자기자본배율,총자산영업이익률,총자산회전율,log총자산,log총자산회전률,자본/부채,이익잉여금/총자산,key,부도,종가변동률
18949,(주)한프,66110.0,2000/12,6.0,0,79.84,60.53,214.52,285.59,-9.47,...,87.84,0.178461,0.688202,10.002065,-0.373673,0.280204,-0.067189,661102001,1.0,-0.339255
18950,(주)한프,66110.0,2001/12,6.0,0,110.35,78.99,117.18,149.71,4.35,...,170.24,0.308199,1.094044,10.170686,0.089881,0.558552,0.116704,661102002,1.0,1.008734
18951,(주)한프,66110.0,2002/12,6.0,0,228.41,187.35,27.82,68.23,27.11,...,425.16,0.267886,0.829631,10.665554,-0.186774,3.148901,0.226560,661102003,1.0,1.008734
18952,(주)한프,66110.0,2003/12,6.0,0,338.80,280.97,19.92,53.29,39.39,...,598.29,0.300043,0.863590,10.920203,-0.146657,4.811206,0.415198,661102004,1.0,1.008734
18953,(주)한프,66110.0,2004/12,6.0,0,347.64,228.08,14.01,67.48,29.86,...,683.95,0.212011,0.845363,11.015246,-0.167989,6.176603,0.516548,661102005,1.0,5.666667
18954,(주)한프,66110.0,2005/12,6.0,0,227.64,154.75,23.05,85.94,21.25,...,724.85,0.085179,0.671062,11.248230,-0.398893,2.604189,0.449964,661102006,1.0,-0.339255
18955,(주)한프,66110.0,2006/12,6.0,0,235.69,157.96,24.89,77.97,24.72,...,732.82,0.076292,0.689843,11.246352,-0.371291,2.729490,0.484817,661102007,1.0,-0.339255
18956,(주)한프,66110.0,2007/12,6.0,0,199.39,131.21,30.02,77.37,21.74,...,680.14,-0.029090,0.657768,11.176039,-0.418903,2.686442,0.463690,661102008,1.0,-0.477593
18957,(주)한프,66110.0,2008/12,6.0,0,128.87,86.05,75.17,98.87,11.09,...,486.45,0.029268,0.709036,11.196075,-0.343849,1.044400,0.251071,661102009,1.0,-0.573529
18958,(주)한프,66110.0,2009/12,6.0,0,160.87,114.84,46.53,75.85,18.79,...,574.38,0.167807,1.035549,11.100723,0.034931,1.972261,0.377777,661102010,1.0,-0.600000


In [96]:
merged = pd.merge(df, price[['key','종가변동률']], on = 'key', how='outer').dropna()
merged

,회사명,거래소코드,회계년도,소속코드,상장폐지일,유동비율,당좌비율,유동부채비율,비유동비율,순운전자본비율,...,자기자본배율,총자산영업이익률,총자산회전율,log총자산,log총자산회전률,자본/부채,이익잉여금/총자산,key,부도,종가변동률
0,(주)CMG제약,58820.0,2000/12,5.0,0,141.01,19.89,131.99,51.76,22.76,...,179.37,0.052949,1.761228,10.274706,0.566011,0.801243,0.189100,588202001,1.0,-0.633803
1,(주)CMG제약,58820.0,2001/12,5.0,0,141.01,19.89,131.99,51.76,22.76,...,179.37,0.014062,0.756792,9.947887,-0.278667,0.725225,0.005261,588202002,1.0,-0.633803
2,(주)CMG제약,58820.0,2002/12,5.0,0,114.21,16.60,132.37,84.50,7.98,...,133.37,-0.247841,1.713785,9.642188,0.538704,0.737002,-0.139212,588202003,1.0,-0.633803
3,(주)CMG제약,58820.0,2003/12,5.0,0,185.63,23.55,71.46,39.31,35.58,...,131.30,-0.082618,1.571816,9.311361,0.452232,1.389417,-0.203019,588202004,1.0,0.538462
4,(주)CMG제약,58820.0,2004/12,5.0,0,149.73,17.11,101.56,50.23,24.97,...,52.25,-0.547982,2.526356,8.552367,0.926778,0.977472,-1.181502,588202005,1.0,-0.312500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27354,흥구석유(주),24060.0,2016/12,5.0,0,411.65,341.54,5.19,93.01,14.15,...,4611.84,0.012181,1.817620,11.278936,0.597528,6.945582,0.414800,240602017,1.0,0.126645
27355,흥구석유(주),24060.0,2017/12,5.0,0,177.50,146.69,12.83,91.80,8.68,...,4650.29,0.009398,1.946103,11.288681,0.665829,6.866903,0.417422,240602018,1.0,-0.189781
27356,흥구석유(주),24060.0,2018/12,5.0,0,329.17,290.09,5.40,89.00,11.59,...,4696.15,0.014520,2.210036,11.227986,0.793009,14.789418,0.453302,240602019,1.0,-0.025225
27357,흥구석유(주),24060.0,2019/12,5.0,0,485.39,422.76,6.05,79.27,21.47,...,5133.42,0.005988,1.800947,11.334528,0.588312,11.561712,0.485889,240602020,1.0,0.981516


In [97]:
merged.isna().sum()

회사명          0
거래소코드        0
회계년도         0
소속코드         0
상장폐지일        0
            ..
자본/부채        0
이익잉여금/총자산    0
key          0
부도           0
종가변동률        0
Length: 70, dtype: int64

In [99]:
for i in merged.columns:
    print(merged[i].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


## 데이터 저장

In [101]:
merged.to_csv('dataset+주가변동률.csv')